# Groupe 6 : évaluation de caractères fake news de messages sur les réseaux sociaux

Le but de ce projet est, à partir d'un ensemble de tweets, d'établir une liste de tweets dont il faut vérifier l'information, triée par ordre de priorité.

## Installations nécessaires :

In [1]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
!python3 -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 23.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
import nltk
nltk.download("punkt") #gestion de la ponctuation pour la tokenization.
nltk.download('vader_lexicon') #lexique de nltk pour la positivité des mots.

[nltk_data] Downloading package punkt to /home/arthur/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/arthur/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

## Lecture de l'ensemble d'entraînement et de test

In [4]:
from src.read.ReadData import ReadData

reader = ReadData(train_path = "data/train.csv",test_path = "data/test.csv")
df_train = reader.read_train()
df_test = reader.read_test()
display(df_train)
display(df_test)

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
7608,Two giant cranes holding a bridge collapse int...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,Police investigating after an e-bike collided ...,1


,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan
...,...
3258,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,Storm in RI worse than last hurricane. My city...
3260,Green Line derailment in Chicago http://t.co/U...
3261,MEG issues Hazardous Weather Outlook (HWO) htt...


## Extraction des caractéristiques :

### Longueur des tweets :

In [5]:
# Test sur 1 tweet :
from src.features.tweetLevel import tweetLevel
from src.features.tokenization import tokenization

extractor_features = tweetLevel()
tokenizer = tokenization()

tweet = df_train["text"][0]
tokenized_tweet = tokenizer.tokenize_tweet(tweet)
len_in_char = extractor_features.get_length_in_characters(tweet)
len_in_tokens = extractor_features.get_length_in_tokens(tweet)

print(f"Tweet : '{tweet}'")
print(f"Longueur en caractères : {len_in_char}")
print("")
print(f"Représentation en tokens : {tokenized_tweet}")
print(f"Longueur en tokens : {len_in_tokens}")

/home/arthur/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Tweet : 'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'
Longueur en caractères : 69

Représentation en tokens : ['Our', 'Deeds', 'are', 'the', 'Reason', 'of', 'this', '#', 'earthquake', 'May', 'ALLAH', 'Forgive', 'us', 'all']
Longueur en tokens : 14


### Sentiment du tweet

In [6]:
sentiment_analysis = extractor_features.get_positive_sentiment_score(tweet)

print(f"Tweet : '{tweet}'")
print(f"Score de positivité du tweet : {sentiment_analysis}")

Tweet : 'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'
Score de positivité du tweet : 0.149


### POS tags du tweet

In [7]:
pos_tags = extractor_features.get_pos_tags(tweet)
print(f"Tweet : '{tweet}'")
print(f"Représentation en tokens : {tokenized_tweet}")
print(f"pos_tags : {pos_tags}")

Tweet : 'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'
Représentation en tokens : ['Our', 'Deeds', 'are', 'the', 'Reason', 'of', 'this', '#', 'earthquake', 'May', 'ALLAH', 'Forgive', 'us', 'all']
pos_tags : [('Our', 'PRP$'), ('Deeds', 'NNS'), ('are', 'VBP'), ('the', 'DT'), ('Reason', 'NNP'), ('of', 'IN'), ('this', 'DT'), ('#', '#'), ('earthquake', 'NN'), ('May', 'NNP'), ('ALLAH', 'NNP'), ('Forgive', 'NNP'), ('us', 'PRP'), ('all', 'DT')]


### Entités du tweet

In [8]:
entity_types = extractor_features.get_entity_types(tweet)
print(f"Tweet : '{tweet}'")
print(f"Entités : {entity_types}")

Tweet : 'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'
Entités : [('May ALLAH Forgive', 'ORG')]


### Word2Vec

In [9]:
from src.features.word2vec import word2vec

train_set = ["Very beautiful","I would like to eat apple","Yes"]
word2vec_creator = word2vec(train_set)
word2vec_creator.predict("beautiful apple")

[array([ 8.1681199e-03, -4.4430327e-03,  8.9854337e-03,  8.2536647e-03,
        -4.4352221e-03,  3.0310510e-04,  4.2744912e-03, -3.9263200e-03,
        -5.5599655e-03, -6.5123225e-03, -6.7073823e-04, -2.9592158e-04,
         4.4630850e-03, -2.4740540e-03, -1.7260908e-04,  2.4618758e-03,
         4.8675989e-03, -3.0808449e-05, -6.3394094e-03, -9.2608072e-03,
         2.6657581e-05,  6.6618943e-03,  1.4660227e-03, -8.9665223e-03,
        -7.9386048e-03,  6.5519023e-03, -3.7856805e-03,  6.2549924e-03,
        -6.6810320e-03,  8.4796622e-03, -6.5163244e-03,  3.2880199e-03,
        -1.0569858e-03, -6.7875278e-03, -3.2875966e-03, -1.1614120e-03,
        -5.4709399e-03, -1.2113475e-03, -7.5633135e-03,  2.6466595e-03,
         9.0701487e-03, -2.3772502e-03, -9.7651005e-04,  3.5135616e-03,
         8.6650876e-03, -5.9218528e-03, -6.8875779e-03, -2.9329848e-03,
         9.1476962e-03,  8.6626766e-04, -8.6784009e-03, -1.4469790e-03,
         9.4794659e-03, -7.5494875e-03, -5.3580985e-03,  9.31656